# Memory

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
from langchain_core.chat_history import BaseChatMessageHistory

class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []

    def add_messages(self, messages):
        self.messages.extend(messages)

    def clear(self):
        self.messages = []

    def __repr__(self):
        return str(self.messages)

In [14]:
store = {}    # item(key=session_id, value=InMemoryHistory_인스턴스)

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [15]:
history_test = get_by_session_id('test')
history_test.add_messages(['hello', 'good morning', 'how are you?'])
history_test.add_messages(['I am fine', 'Thank you'])

history_test

['hello', 'good morning', 'how are you?', 'I am fine', 'Thank you']

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}을 잘하는 AI 어시스턴트야.'),
    MessagesPlaceholder(variable_name='history'),
    ('human', '{query}')
])
model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.5)
chain = prompt | model

In [17]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'
)

In [18]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '다람쥐는 도토리 나무를 세 그루 키우고 있습니다.'},
    config={'configurable': {'session_id': 'squirrel'}}
)

print(response)

content='다람쥐가 도토리 나무를 세 그루 키우고 있다니 정말 흥미로운 이야기네요! 도토리 나무는 다람쥐에게 중요한 식량 공급원이죠. 그들은 도토리를 모아서 겨울철에 먹기도 하고, 나무를 심어 새로운 나무가 자라도록 돕기도 해요. 다람쥐가 키우는 도토리 나무에 대해 더 이야기해 줄 수 있을까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 46, 'total_tokens': 151, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-c469a339-c283-44a8-9097-a7a44a3cb82e-0' usage_metadata={'input_tokens': 46, 'output_tokens': 105, 'total_tokens': 151, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [19]:
store

{'test': ['hello', 'good morning', 'how are you?', 'I am fine', 'Thank you'],
 'squirrel': [HumanMessage(content='다람쥐는 도토리 나무를 세 그루 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='다람쥐가 도토리 나무를 세 그루 키우고 있다니 정말 흥미로운 이야기네요! 도토리 나무는 다람쥐에게 중요한 식량 공급원이죠. 그들은 도토리를 모아서 겨울철에 먹기도 하고, 나무를 심어 새로운 나무가 자라도록 돕기도 해요. 다람쥐가 키우는 도토리 나무에 대해 더 이야기해 줄 수 있을까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 46, 'total_tokens': 151, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-c469a339-c283-44a8-9097-a7a44a3cb82e-0', usage_metadata={'input_tokens': 46, 'output_tokens': 105, 'total_tokens': 151, 'input_token_details': {'audio': 

In [20]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '토끼는 사과 나무를 다섯 그루 키우고 있습니다.'},
    config={'configurable': {'session_id': 'squirrel'}}
)

print(response)

content='토끼가 사과 나무를 다섯 그루 키우고 있다니 참 귀엽고 흥미로운 설정이네요! 사과 나무는 토끼에게 맛있는 과일을 제공할 뿐만 아니라, 그늘도 만들어 주어 편안한 장소가 될 수 있겠죠. 토끼는 사과를 좋아하니까, 나무에서 자란 사과를 맛있게 먹을 수 있을 것 같아요. 토끼가 키우는 사과 나무에 대한 이야기가 더 있나요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 176, 'total_tokens': 291, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-77702aa1-7c14-4c38-b291-c9182cfa36da-0' usage_metadata={'input_tokens': 176, 'output_tokens': 115, 'total_tokens': 291, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [21]:
store

{'test': ['hello', 'good morning', 'how are you?', 'I am fine', 'Thank you'],
 'squirrel': [HumanMessage(content='다람쥐는 도토리 나무를 세 그루 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='다람쥐가 도토리 나무를 세 그루 키우고 있다니 정말 흥미로운 이야기네요! 도토리 나무는 다람쥐에게 중요한 식량 공급원이죠. 그들은 도토리를 모아서 겨울철에 먹기도 하고, 나무를 심어 새로운 나무가 자라도록 돕기도 해요. 다람쥐가 키우는 도토리 나무에 대해 더 이야기해 줄 수 있을까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 46, 'total_tokens': 151, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-c469a339-c283-44a8-9097-a7a44a3cb82e-0', usage_metadata={'input_tokens': 46, 'output_tokens': 105, 'total_tokens': 151, 'input_token_details': {'audio': 

In [22]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '다람쥐와 토끼는 합해서 몇 그루의 나무를 키우고 있나요?'},
    config={'configurable': {'session_id': 'squirrel'}}
)

print(response)

content='다람쥐가 도토리 나무를 세 그루, 토끼가 사과 나무를 다섯 그루 키우고 있으니, 합쳐서 총 8그루의 나무를 키우고 있는 셈이네요! (3 + 5 = 8) 정말 멋진 공동체를 이루고 있겠어요!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 322, 'total_tokens': 401, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None} id='run-52026734-8da0-4437-b898-47182b8d2e1e-0' usage_metadata={'input_tokens': 322, 'output_tokens': 79, 'total_tokens': 401, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
